# Satellite Imagery-Based Property Valuation: High-Performance Model

This notebook implements the complete pipeline:
1.  **Tabular Baseline:** XGBoost on ALL features (Including Lat/Long).
2.  **Multimodal Network:** ResNet18 + MLP (with Location Features).
3.  **Visual Forensics:** Grad-CAM Analysis.

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Load Data & Baseline Model
**Performance Boost:** We are now including `Lat`, `Long`, and `Zipcode`. Location is the most important factor in real estate.

In [ ]:
# Load csv
DATA_PATH = '../data/train.csv' 
if not os.path.exists(DATA_PATH):
    DATA_PATH = 'data/train.csv' # Fallback for local run

df = pd.read_csv(DATA_PATH)
print(f"Loaded {len(df)} rows")

# --- FEATURE ENGINEERING ---
# 1. Handle Date
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# 2. Select Features (Include Lat/Long this time!)
# We only drop ID and PRICE. Everything else is useful.
exclude_cols = ['id', 'price', 'date'] 
feature_cols = [c for c in df.columns if c not in exclude_cols]
print(f"Training on {len(feature_cols)} features: {feature_cols}")

X = df[feature_cols].fillna(0)
y = df['price']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Baseline (XGBoost/GBR)
print("Training Tabular Baseline (with Location features)...")
baseline_model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, max_depth=5, random_state=42)
baseline_model.fit(X_train, y_train)

# Evaluate Baseline
preds = baseline_model.predict(X_val)
baseline_rmse = np.sqrt(mean_squared_error(y_val, preds))
baseline_r2 = r2_score(y_val, preds)

print(f"Tabular Baseline RMSE: ${baseline_rmse:,.2f}")
print(f"Tabular Baseline R2 Score: {baseline_r2:.4f}")

## 2. Multimodal Deep Learning
Training on Log(Price) for stability.

In [ ]:
# Dataset & Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class RealEstateDataset(Dataset):
    def __init__(self, tabular_data, image_dir, feature_cols, scaler=None, transform=None):
        self.data = tabular_data.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        self.feature_cols = feature_cols
        self.num_features = len(self.feature_cols)
        
        if scaler:
            self.X_scaled = scaler.transform(self.data[self.feature_cols].fillna(0))
        else:
            self.X_scaled = self.data[self.feature_cols].fillna(0).values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.X_scaled[idx].astype(np.float32))
        
        # Log-Price Target
        raw_price = self.data.loc[idx, 'price']
        log_price = np.log1p(raw_price)
        target = torch.tensor(log_price, dtype=torch.float32)
        
        img_id = str(int(self.data.loc[idx, 'id']))
        img_path = os.path.join(self.image_dir, f"{img_id}.jpg")
        
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224), color='black')
            
        if self.transform:
            image = self.transform(image)
            
        return image, features, target

In [ ]:
# Model Architecture
class MultimodalNet(nn.Module):
    def __init__(self, num_tabular_features):
        super(MultimodalNet, self).__init__()
        self.cnn = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.cnn.fc = nn.Identity()
        
        self.mlp = nn.Sequential(
            nn.Linear(num_tabular_features, 128), # Wider for more features
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        
        self.fusion = nn.Sequential(
            nn.Linear(512 + 64, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 1)
        )

    def forward(self, image, tabular):
        x_img = self.cnn(image)
        x_tab = self.mlp(tabular)
        combined = torch.cat((x_img, x_tab), dim=1)
        return self.fusion(combined)

## 3. Training Loop & Evaluation

In [ ]:
def train_and_evaluate(epochs=15):
    # Configuration
    IMG_DIR = '../data/images' 
    if not os.path.exists(IMG_DIR): IMG_DIR = 'data/images'
        
    train_split, val_split = train_test_split(df, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    scaler.fit(train_split[feature_cols].fillna(0))
    
    train_dataset = RealEstateDataset(train_split, IMG_DIR, feature_cols, scaler, transform=transform)
    val_dataset = RealEstateDataset(val_split, IMG_DIR, feature_cols, scaler, transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    model = MultimodalNet(num_tabular_features=train_dataset.num_features).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
    criterion = nn.MSELoss()

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)
    
    print(f"Starting Training on {len(train_dataset)} samples...")
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, tabs, log_prices in tqdm(train_loader):
            images = images.to(device)
            tabs = tabs.to(device)
            log_prices = log_prices.to(device).unsqueeze(1)
            
            optimizer.zero_grad()
            outputs = model(images, tabs)
            loss = criterion(outputs, log_prices)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        avg_loss = running_loss/len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss (Log-MSE): {avg_loss:.4f}")
        scheduler.step(avg_loss)
        
    print("\nEvaluating on Validation Set...")
    model.eval()
    all_preds_log = []
    all_targets_log = []
    
    with torch.no_grad():
        for images, tabs, log_prices in val_loader:
            images = images.to(device)
            tabs = tabs.to(device)
            
            outputs = model(images, tabs)
            all_preds_log.extend(outputs.cpu().numpy())
            all_targets_log.extend(log_prices.numpy())
            
    pred_prices = np.expm1(np.array(all_preds_log).flatten())
    real_prices = np.expm1(np.array(all_targets_log).flatten())
    
    dl_rmse = np.sqrt(mean_squared_error(real_prices, pred_prices))
    dl_r2 = r2_score(real_prices, pred_prices)
    
    print("="*40)
    print(f"Multimodal Model RMSE: ${dl_rmse:,.2f}")
    print(f"Multimodal Model R2:   {dl_r2:.4f}")
    print("="*40)
    
    return model

# Run Training
trained_model = train_and_evaluate(epochs=15)

## 4. Visual Analysis (Grad-CAM)
This section visualizes **which parts of the image** the model looks at.

In [ ]:
def analyze_visual_features(model, img_path):
    img_pil = Image.open(img_path).convert('RGB')
    img_tensor = transform(img_pil).unsqueeze(0).to(device)
    
    activations = []
    gradients = []
    
    def hook_forward(module, input, output):
        activations.append(output)
    def hook_backward(module, grad_in, grad_out):
        gradients.append(grad_out[0])
        
    handle_f = model.cnn.layer4[-1].register_forward_hook(hook_forward)
    handle_b = model.cnn.layer4[-1].register_full_backward_hook(hook_backward)
    
    model.eval()
    # Dynamic Size Fix
    num_input_features = model.mlp[0].in_features
    dummy_tab = torch.zeros((1, num_input_features)).to(device)
    
    output = model(img_tensor, dummy_tab)
    model.zero_grad()
    output.backward()
    
    pooled_gradients = torch.mean(gradients[0], dim=[0, 2, 3])
    activation = activations[0][0]
    for i in range(512):
        activation[i, :, :] *= pooled_gradients[i]
        
    heatmap = torch.mean(activation, dim=0).cpu().detach().numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    
    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    original_img = cv2.cvtColor(np.array(img_pil.resize((224, 224))), cv2.COLOR_RGB2BGR)
    superimposed = cv2.addWeighted(original_img, 0.6, heatmap, 0.4, 0)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1); plt.title("Original"); plt.imshow(img_pil)
    plt.subplot(1, 2, 2); plt.title("Model Attention"); plt.imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
    plt.show()
    
    handle_f.remove()
    handle_b.remove()

# Example Usage:
# analyze_visual_features(trained_model, '../data/images/some_id.jpg')